Evaluate the performance of different types of optimizer on a LeNet-5 network using MNIST data. At least you need to evaluate SGD, AdaGrad, RMSprop. 

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

In [2]:
import numpy as np
from datetime import datetime 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

import matplotlib.pyplot as plt

# check device
#DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# Preparing for Data
print('==> Preparing data..')

"""
# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

"""

==> Preparing data..


"\n# Training Data augmentation\ntransform_train = transforms.Compose([\n    transforms.RandomCrop(32, padding=4),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n# Testing Data preparation\ntransform_test = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n\n#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')\n\n"

In [4]:
# Preparing for Data
print('==> Preparing data..')

# define transforms
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])

==> Preparing data..


In [5]:
#Defining the convolutional neural network
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = torch.flatten(out, 1) # flatten all dimensions except batch
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
 
        return out

In [6]:
model1 = LeNet()

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model1.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model1):,} trainable parameters')

The model has 61,706 trainable parameters


In [8]:
########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

#import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [9]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    count = 0
    train_loss = 0
    total = 0 
    correct = 0
    
    total_step = len(train_loader)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        

        #forward pass
        y_pred = model(data)

        loss = criterion(y_pred, target)

        acc = calculate_accuracy(y_pred, target)
        
        
        #Backward pass
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

In [10]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [12]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='mnist_data', train=True, download=True, transform=transforms)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='mnist_data', train=False, download=True, transform=transforms)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.Adagrad(model.parameters(), lr=lr, lr_decay=0, weight_decay=5e-4, initial_accumulator_value=0, eps=1e-10)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298496
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.057791
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.410055
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.663376
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.595390
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.452076
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.331029
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.395366
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.330915
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.194199
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.208472
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.358802
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.107097
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.306074
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.218555
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.137045
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.109381
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.110605
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.115708
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.200299
Train Epoch: 1 [

Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.056407
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.062026
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.038929
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.049893
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.083826
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.048905
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.037353
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.022927
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.058964
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.039561
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.093861
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.039437
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.068801
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.043497
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.019620
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.076583
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.036406
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.050515
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.010529
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.053947


Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.042829
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.058275
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.019588
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.031512
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.015752

Test set: Average loss: -12.0543, Accuracy: 9874/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.063148
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.007767
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.019237
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.011236
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.013300
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.017509
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.050070
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.023893
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.033335
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.037325
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.037255
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.037664
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.010931
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.024412
Tra

Train Epoch: 11 [16640/60000 (28%)]	Loss: 0.029811
Train Epoch: 11 [17920/60000 (30%)]	Loss: 0.023179
Train Epoch: 11 [19200/60000 (32%)]	Loss: 0.050534
Train Epoch: 11 [20480/60000 (34%)]	Loss: 0.020828
Train Epoch: 11 [21760/60000 (36%)]	Loss: 0.010278
Train Epoch: 11 [23040/60000 (38%)]	Loss: 0.031311
Train Epoch: 11 [24320/60000 (41%)]	Loss: 0.009675
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.016388
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.047400
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.012533
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.013297
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.018600
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.029477
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.011740
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.030510
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.008784
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.015375
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.013751
Train Epoch: 11 [39680/60000 (66%)]	Loss: 0.003885
Train Epoch: 11 [40960/60000 (6

Train Epoch: 14 [38400/60000 (64%)]	Loss: 0.032675
Train Epoch: 14 [39680/60000 (66%)]	Loss: 0.013961
Train Epoch: 14 [40960/60000 (68%)]	Loss: 0.007636
Train Epoch: 14 [42240/60000 (70%)]	Loss: 0.022017
Train Epoch: 14 [43520/60000 (72%)]	Loss: 0.002313
Train Epoch: 14 [44800/60000 (75%)]	Loss: 0.008910
Train Epoch: 14 [46080/60000 (77%)]	Loss: 0.020499
Train Epoch: 14 [47360/60000 (79%)]	Loss: 0.005969
Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.012173
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.013793
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.058229
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.016986
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.009116
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.016092
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.026142
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.054018
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.004910

Test set: Average loss: -13.3180, Accuracy: 9898/10000 (99%)

Train Epoch: 15 [0/60000 (0%)]	Loss: 0.026023
Train Epoch: 15 [1280/60

...

Train Epoch: 34 [47360/60000 (79%)]	Loss: 0.025823
Train Epoch: 34 [48640/60000 (81%)]	Loss: 0.007622
Train Epoch: 34 [49920/60000 (83%)]	Loss: 0.005581
Train Epoch: 34 [51200/60000 (85%)]	Loss: 0.005639
Train Epoch: 34 [52480/60000 (87%)]	Loss: 0.008133
Train Epoch: 34 [53760/60000 (90%)]	Loss: 0.017616
Train Epoch: 34 [55040/60000 (92%)]	Loss: 0.008914
Train Epoch: 34 [56320/60000 (94%)]	Loss: 0.033164
Train Epoch: 34 [57600/60000 (96%)]	Loss: 0.007986
Train Epoch: 34 [58880/60000 (98%)]	Loss: 0.008134

Test set: Average loss: -14.5645, Accuracy: 9894/10000 (99%)

Train Epoch: 35 [0/60000 (0%)]	Loss: 0.011289
Train Epoch: 35 [1280/60000 (2%)]	Loss: 0.012385
Train Epoch: 35 [2560/60000 (4%)]	Loss: 0.001090
Train Epoch: 35 [3840/60000 (6%)]	Loss: 0.024815
Train Epoch: 35 [5120/60000 (9%)]	Loss: 0.000847
Train Epoch: 35 [6400/60000 (11%)]	Loss: 0.001632
Train Epoch: 35 [7680/60000 (13%)]	Loss: 0.008007
Train Epoch: 35 [8960/60000 (15%)]	Loss: 0.003280
Train Epoch: 35 [10240/60000 (17%)]

Train Epoch: 38 [7680/60000 (13%)]	Loss: 0.006052
Train Epoch: 38 [8960/60000 (15%)]	Loss: 0.012005
Train Epoch: 38 [10240/60000 (17%)]	Loss: 0.006795
Train Epoch: 38 [11520/60000 (19%)]	Loss: 0.009985
Train Epoch: 38 [12800/60000 (21%)]	Loss: 0.005130
Train Epoch: 38 [14080/60000 (23%)]	Loss: 0.019070
Train Epoch: 38 [15360/60000 (26%)]	Loss: 0.006798
Train Epoch: 38 [16640/60000 (28%)]	Loss: 0.006282
Train Epoch: 38 [17920/60000 (30%)]	Loss: 0.004200
Train Epoch: 38 [19200/60000 (32%)]	Loss: 0.008043
Train Epoch: 38 [20480/60000 (34%)]	Loss: 0.006990
Train Epoch: 38 [21760/60000 (36%)]	Loss: 0.013342
Train Epoch: 38 [23040/60000 (38%)]	Loss: 0.018525
Train Epoch: 38 [24320/60000 (41%)]	Loss: 0.012276
Train Epoch: 38 [25600/60000 (43%)]	Loss: 0.001591
Train Epoch: 38 [26880/60000 (45%)]	Loss: 0.004725
Train Epoch: 38 [28160/60000 (47%)]	Loss: 0.005444
Train Epoch: 38 [29440/60000 (49%)]	Loss: 0.001569
Train Epoch: 38 [30720/60000 (51%)]	Loss: 0.006997
Train Epoch: 38 [32000/60000 (53%

Train Epoch: 41 [29440/60000 (49%)]	Loss: 0.003826
Train Epoch: 41 [30720/60000 (51%)]	Loss: 0.011813
Train Epoch: 41 [32000/60000 (53%)]	Loss: 0.004224
Train Epoch: 41 [33280/60000 (55%)]	Loss: 0.006694
Train Epoch: 41 [34560/60000 (58%)]	Loss: 0.005125
Train Epoch: 41 [35840/60000 (60%)]	Loss: 0.007241
Train Epoch: 41 [37120/60000 (62%)]	Loss: 0.004036
Train Epoch: 41 [38400/60000 (64%)]	Loss: 0.002548
Train Epoch: 41 [39680/60000 (66%)]	Loss: 0.008612
Train Epoch: 41 [40960/60000 (68%)]	Loss: 0.016390
Train Epoch: 41 [42240/60000 (70%)]	Loss: 0.008171
Train Epoch: 41 [43520/60000 (72%)]	Loss: 0.002062
Train Epoch: 41 [44800/60000 (75%)]	Loss: 0.013863
Train Epoch: 41 [46080/60000 (77%)]	Loss: 0.002918
Train Epoch: 41 [47360/60000 (79%)]	Loss: 0.002618
Train Epoch: 41 [48640/60000 (81%)]	Loss: 0.003974
Train Epoch: 41 [49920/60000 (83%)]	Loss: 0.004564
Train Epoch: 41 [51200/60000 (85%)]	Loss: 0.019637
Train Epoch: 41 [52480/60000 (87%)]	Loss: 0.015181
Train Epoch: 41 [53760/60000 (9

Train Epoch: 44 [51200/60000 (85%)]	Loss: 0.036418
Train Epoch: 44 [52480/60000 (87%)]	Loss: 0.012253
Train Epoch: 44 [53760/60000 (90%)]	Loss: 0.003945
Train Epoch: 44 [55040/60000 (92%)]	Loss: 0.003948
Train Epoch: 44 [56320/60000 (94%)]	Loss: 0.010278
Train Epoch: 44 [57600/60000 (96%)]	Loss: 0.003773
Train Epoch: 44 [58880/60000 (98%)]	Loss: 0.063068

Test set: Average loss: -15.1061, Accuracy: 9904/10000 (99%)

Train Epoch: 45 [0/60000 (0%)]	Loss: 0.015247
Train Epoch: 45 [1280/60000 (2%)]	Loss: 0.012261
Train Epoch: 45 [2560/60000 (4%)]	Loss: 0.006660
Train Epoch: 45 [3840/60000 (6%)]	Loss: 0.003773
Train Epoch: 45 [5120/60000 (9%)]	Loss: 0.006666
Train Epoch: 45 [6400/60000 (11%)]	Loss: 0.005709
Train Epoch: 45 [7680/60000 (13%)]	Loss: 0.011904
Train Epoch: 45 [8960/60000 (15%)]	Loss: 0.011805
Train Epoch: 45 [10240/60000 (17%)]	Loss: 0.005203
Train Epoch: 45 [11520/60000 (19%)]	Loss: 0.006385
Train Epoch: 45 [12800/60000 (21%)]	Loss: 0.005291
Train Epoch: 45 [14080/60000 (23%)]

Train Epoch: 48 [11520/60000 (19%)]	Loss: 0.007786
Train Epoch: 48 [12800/60000 (21%)]	Loss: 0.015346
Train Epoch: 48 [14080/60000 (23%)]	Loss: 0.001692
Train Epoch: 48 [15360/60000 (26%)]	Loss: 0.007540
Train Epoch: 48 [16640/60000 (28%)]	Loss: 0.024321
Train Epoch: 48 [17920/60000 (30%)]	Loss: 0.014428
Train Epoch: 48 [19200/60000 (32%)]	Loss: 0.000662
Train Epoch: 48 [20480/60000 (34%)]	Loss: 0.017541
Train Epoch: 48 [21760/60000 (36%)]	Loss: 0.003801
Train Epoch: 48 [23040/60000 (38%)]	Loss: 0.002789
Train Epoch: 48 [24320/60000 (41%)]	Loss: 0.007242
Train Epoch: 48 [25600/60000 (43%)]	Loss: 0.004345
Train Epoch: 48 [26880/60000 (45%)]	Loss: 0.003848
Train Epoch: 48 [28160/60000 (47%)]	Loss: 0.013111
Train Epoch: 48 [29440/60000 (49%)]	Loss: 0.016570
Train Epoch: 48 [30720/60000 (51%)]	Loss: 0.018861
Train Epoch: 48 [32000/60000 (53%)]	Loss: 0.005155
Train Epoch: 48 [33280/60000 (55%)]	Loss: 0.003969
Train Epoch: 48 [34560/60000 (58%)]	Loss: 0.012180
Train Epoch: 48 [35840/60000 (6